# Importing libraries

All of the necessary libraries are importd to perform various functions upon the dataset

In [322]:
# linear algebra
import numpy as np 

#Import os
import os

# data processing
import pandas as pd

In [323]:
# data visualization
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

In [324]:
# Machine learning algorithms
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [325]:
# Library to split the dataset
from sklearn.model_selection import train_test_split

In [326]:
# Storing the titanic datset within a dataframe
# filename = 'titanic.csv'
# filename = 'kaggle_train.csv'
filename = 'kaggle_test.csv'

complete_df = pd.read_csv(os.path.join('..','..','data',filename))

# Data Analysis

Here the dataset is analysed to see what exactly it is made up of.

In [327]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [328]:
complete_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [329]:
complete_df.head(8)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S


Below is a graph depcting the survival rate of people according to their gender and their passenger class. It clearly indicates that females and people from higher classes were most likely to survive

In [330]:

try:
    #Creating the bar plot grouped across classes
    sns.barplot(x = 'Sex',y = 'Survived',hue = 'Pclass',data = complete_df, palette = "Blues")
    #Adding the aesthetics
    plt.title('Gender vs Survival')
    plt.xlabel('Gender')
    plt.ylabel('Survival Rate') 
    # Show the plot
    plt.show()
except:
    print('Skipping')

Skipping


# Managing the missing data

Showing the missing data within the data set

In [331]:
total = complete_df.isnull().sum().sort_values(ascending=False)
percent_1 = complete_df.isnull().sum()/complete_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(10)

,Total,%
Cabin,327,78.2
Age,86,20.6
Fare,1,0.2
PassengerId,0,0.0
Pclass,0,0.0
Name,0,0.0
Sex,0,0.0
SibSp,0,0.0
Parch,0,0.0
Ticket,0,0.0


##### Removing unnecessary variables to simplify the data being used to generate the predictive model

In [332]:
# Removing the cabin feature as it contains too many missing variables
complete_df = complete_df.drop(['Cabin'], axis=1)
complete_df = complete_df.drop(['Ticket'], axis=1)

######  Imputing the missing value in column with the average

In [333]:
complete_df['Fare'].fillna(value=complete_df['Fare'].mean(), inplace=True)

In [334]:
complete_df = complete_df.dropna(subset=['Fare'])
complete_df[complete_df['Fare'].isna()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked


###### Filling in the missing values for the embarked feature

In [335]:
complete_df['Embarked'].describe()

count     418
unique      3
top         S
freq      270
Name: Embarked, dtype: object

In [336]:
complete_df['Embarked'].fillna("S", inplace = True)

In [337]:
complete_df['Embarked'].describe()

count     418
unique      3
top         S
freq      270
Name: Embarked, dtype: object

# Creating and altering features

###### Adding a feature to represent the title that each passenger possessed to help further differentiate passengers even within their passenger classes

In [338]:
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
data = [complete_df]

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)
complete_df = complete_df.drop(['Name'], axis=1)

Below is a graph depcting the title vs survival rate of the people with the different hues of red depicting the different passenger classes within each category

In [339]:
try:
    #Creating the bar plot grouped across classes
    sns.barplot(x = 'Title',y = 'Survived',hue = 'Pclass',data = complete_df, palette = "Reds")
    #Adding the aesthetics
    plt.title('Title vs Survival')
    plt.xlabel('Title')
    plt.ylabel('Survival Rate') 
    # Show the plot
    plt.show()

except:
    print('Skipping')

Skipping


###### Filling in the missing age values by using the mean values for other people who have a similar passenger class, sex and title as them

In [340]:
complete_df['Age'].fillna(complete_df.groupby(['Pclass','Sex','Title'])['Age'].transform('mean'), inplace=True)

Graph representing the amount of people who survived from each age bracket when compared to those who did not survive

In [341]:
try:

    sns.histplot(data=complete_df, stat="count", multiple="stack",
                 x="Age", kde=False,
                 hue="Survived",
                 element="bars", legend=True)
    plt.title("Survival within different age brackets")
    plt.xlabel("Age")
    plt.ylabel("Count")

    plt.show()
except:
    print('Skipping')

Skipping


Converting the categorical features into numerical ones so that they may be accepted as input into the machine learning algorithms

In [342]:
genders = {"male": 0, "female": 1}

data = [complete_df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

###### Changing float features to int

In [343]:
data = [complete_df]

for dataset in data:
    dataset['Fare'] = dataset['Fare'].astype(int)
    dataset['Pclass'] = dataset['Pclass'].astype(int)
    dataset['Sex'] = dataset['Sex'].astype(int)
    dataset['Title'] = dataset['Title'].astype(int)
    dataset['SibSp'] = dataset['SibSp'].astype(int)
    dataset['Parch'] = dataset['Parch'].astype(int)
    
# Make age into speperate features
# Try to see if dropping fare will help
# Try to see which variables affect algorithm results

###### Performing One-hot encoding on class and embarked variables

In [344]:
#First perform the one hot encoding and then remove a single column
complete_df = pd.get_dummies(complete_df, columns = ['Pclass', 'Embarked'])

###### Dropping additional columns

In [345]:
complete_df = complete_df.drop(['Pclass_3'], axis=1)
complete_df = complete_df.drop(['Embarked_S'], axis=1)

In [346]:
complete_df.head(8)

,PassengerId,Sex,Age,SibSp,Parch,Fare,Title,Pclass_1,Pclass_2,Embarked_C,Embarked_Q
0,892,0,34.5,0,0,7,1,0,0,0,1
1,893,1,47.0,1,0,7,3,0,0,0,0
2,894,0,62.0,0,0,9,1,0,1,0,1
3,895,0,27.0,0,0,8,1,0,0,0,0
4,896,1,22.0,1,1,12,3,0,0,0,0
5,897,0,14.0,0,0,9,1,0,0,0,0
6,898,1,30.0,0,0,7,2,0,0,0,1
7,899,0,26.0,1,1,29,1,0,1,0,0


###### Saving the data

In [347]:
# complete_df.to_csv(os.path.join('..','..',"data",'Variant 2',"titanic_cleaned.csv"),index=False)
# complete_df.to_csv(os.path.join('..','..',"data",'Variant 2',"kaggle_train_cleaned.csv"),index=False)
complete_df.to_csv(os.path.join('..','..',"data",'Variant 2',"kaggle_test_cleaned.csv"),index=False)